In [5]:
import os

# Existing function to get all files in a directory
def GetFilesInDirectory(directory):
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            paths.append(file_path)
    return paths

# # Define patterns for different Python application types
# PATTERNS = {
#     'Django': ['manage.py', 'requirements.txt', 'urls.py', 'settings.py', 'wsgi.py'],
#     'Flask': ['app.py', 'requirements.txt', 'static/', 'templates/', 'flask'],
#     'FastAPI': ['main.py', 'requirements.txt', 'fastapi', 'pydantic'],
#     'Data Science': ['notebooks/', 'requirements.txt', 'pandas', 'numpy', 'matplotlib'],
#     'Machine Learning': ['train.py', 'requirements.txt', 'tensorflow', 'scikit-learn', 'pytorch'],
#     'Automation': ['automation.py', 'requirements.txt', 'os', 'requests'],
#     'Web Scraping': ['scraper.py', 'requirements.txt', 'BeautifulSoup', 'Scrapy'],
#     'Tkinter GUI': ['main.py', 'requirements.txt', 'tkinter'],
#     'IoT (Raspberry Pi)': ['main.py', 'requirements.txt', 'RPi.GPIO'],
#     'AWS DevOps': ['aws.py', 'requirements.txt', 'boto3'],
#     'Unit Testing': ['test_main.py', 'requirements.txt', 'pytest', 'unittest'],
#     'Streamlit' : ['app.py', 'requirements.txt', '']
# }

# # Function to check if the directory contains a Python application
# def CheckPythonApplication(directory):
#     files_in_directory = GetFilesInDirectory(directory)
#     print(files_in_directory)
    
#     # Normalize the file paths for easier comparison
#     normalized_files = [os.path.basename(file) for file in files_in_directory]
#     print(normalized_files)
    
#     # Iterate over each pattern and check for matches
#     for app_type, indicators in PATTERNS.items():
#         if all(any(indicator in file for file in normalized_files) for indicator in indicators):
#             return f'Python application detected: {app_type}'
    
#     return 'No Python application detected.'

# # Example usage
# directory_path = '.'  # Replace with the directory path you want to check
# result = CheckPythonApplication(directory_path)
# print(result)


['./checkreq.py', './requirements.txt', './dockerfilegen.ipynb', './Dockerfile', './requirements2.txt', './test_pythondir.ipynb', './utils.py', './sample.py', './dockerfilegen copy.ipynb', './__pycache__/utils.cpython-39.pyc', './__pycache__/checkreq.cpython-39.pyc', './__pycache__/checkreq.cpython-36.pyc', './__pycache__/checkreq.cpython-312.pyc', './__pycache__/checkreq.cpython-311.pyc', './__pycache__/checkreq.cpython-310.pyc', './.git/ORIG_HEAD', './.git/config', './.git/HEAD', './.git/description', './.git/index', './.git/packed-refs', './.git/FETCH_HEAD', './.git/objects/pack/pack-7f5fdf36889833a59560505a585bdbfd8a032744.pack', './.git/objects/pack/pack-7f5fdf36889833a59560505a585bdbfd8a032744.idx', './.git/objects/07/35a283ad04c4b3555346265806ae169da9e2c1', './.git/info/exclude', './.git/logs/HEAD', './.git/logs/refs/heads/dev_ajay', './.git/logs/refs/heads/main', './.git/logs/refs/remotes/origin/dev_ajay', './.git/logs/refs/remotes/origin/HEAD', './.git/logs/refs/remotes/origin

In [1]:
import dspy
from pydantic import BaseModel, Field
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
from utils import GetFilesInDirectory

/opt/anaconda3/envs/storm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bedrock = dspy.Bedrock(region_name="us-east-1")

model_kwargs =  { 
        "max_tokens": 10000,
        "temperature": 0.7,
        # "top_k": 50,
        # "top_p": 0.9
    }

lm = dspy.AWSAnthropic(bedrock, "anthropic.claude-3-5-sonnet-20240620-v1:0", **model_kwargs)

dspy.configure(lm=lm)

In [3]:
import platform
import os

def detect_application(requirements_file, directory_structure):
    # Read the requirements.txt file
    with open(requirements_file, 'r') as file:
        requirements = file.read()

    # Get the Python version
    python_version = platform.python_version()

    # Determine the type of application based on dependencies
    type_of_application = None
    if 'streamlit' in requirements:
        type_of_application = 'Streamlit'
    elif 'langchain' in requirements:
        type_of_application = 'Langchain'
    
    # Look for a common entry point file (app.py or main.py) in the directory structure
    entrypoint_filename = 'app.py'  # Default guess
    for file in directory_structure:
        if file.endswith(('app.py', 'main.py')):
            entrypoint_filename = os.path.basename(file)
            break
    
    # # Set the working directory
    # work_directory = 'src'  # Default work directory
    # for file in directory_structure:
    #     if 'src/' in file:
    #         work_directory = 'src'
    #         break

    class Input(BaseModel):
        dir_struct: List[str] = Field(description="List of all the files in the directory.")
       

    class Output(BaseModel):
        work_directory: str = Field(description="The working directory of the application")


    class FindWorkingDirectory(dspy.Signature):
        f"""I need the working directory for this {type_of_application} application so that I can create a Dockerfile."""

        input: Input = dspy.InputField()
        output: Output = dspy.OutputField()

    predictor = dspy.TypedPredictor(FindWorkingDirectory)
    input = Input(
        dir_struct = directory_structure,
    )
    mapped_parameters = predictor(input=input)
    print(mapped_parameters.output.work_directory)
    


    # Output the final format
    return f'''Input(
    python_version = "{python_version}",
    type_of_application = '{type_of_application}',
    work_directory = '{mapped_parameters.output.work_directory}',
    entrypoint_filename = '{entrypoint_filename}'
)'''


output = detect_application('requirements2.txt', GetFilesInDirectory("."))
print(output)


./src
Input(
    python_version = "3.11.9",
    type_of_application = 'Streamlit',
    work_directory = './src',
    entrypoint_filename = 'app.py'
)
